In [2]:
%pip install pandas
%pip install scikit-learn
%pip install matplotlib
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [16]:
data = pd.read_csv('./HR_capstone_dataset.csv')
data = data[data["left"] == 0]
data

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
2000,0.58,0.74,4,215,3,0,0,0,sales,low
2001,0.82,0.67,2,202,3,0,0,0,sales,low
2002,0.45,0.69,5,193,3,0,0,0,sales,low
2003,0.78,0.82,5,247,3,0,0,0,sales,low
2004,0.49,0.60,3,214,2,0,0,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14206,0.90,0.55,3,259,10,1,0,1,management,high
14207,0.74,0.95,5,266,10,0,0,1,management,high
14208,0.85,0.54,3,185,10,0,0,1,management,high
14209,0.33,0.65,3,172,10,0,0,1,marketing,high


In [17]:
unique_departments = data['Work_accident'].unique()
unique_salary = data['promotion_last_5years'].unique()
print("Unique Departments:", unique_departments)
print("Unique salary:", unique_salary)

Unique Departments: [0 1]
Unique salary: [0 1]


In [6]:
data.drop(data[data['Department'] == 'RandD'].index, inplace=True)
data

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
2000,0.58,0.74,4,215,3,0,0,0,sales,low
2001,0.82,0.67,2,202,3,0,0,0,sales,low
2002,0.45,0.69,5,193,3,0,0,0,sales,low
2003,0.78,0.82,5,247,3,0,0,0,sales,low
2004,0.49,0.60,3,214,2,0,0,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14206,0.90,0.55,3,259,10,1,0,1,management,high
14207,0.74,0.95,5,266,10,0,0,1,management,high
14208,0.85,0.54,3,185,10,0,0,1,management,high
14209,0.33,0.65,3,172,10,0,0,1,marketing,high


In [7]:
categorical_features = ['Department', 'salary']
one_hot_encoder = OneHotEncoder()

In [8]:
preprocessor = ColumnTransformer(
    transformers=[
        ('string', one_hot_encoder, categorical_features)
    ], remainder='passthrough')

In [9]:
X = data.drop('satisfaction_level', axis=1)
y = data['satisfaction_level']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

In [12]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('string', OneHotEncoder(),
                                                  ['Department', 'salary'])])),
                ('regressor', LinearRegression())])

In [13]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 0.04552005769118496


In [14]:
from joblib import dump, load

# Save the model to disk
dump(model, 'employee_satisfaction_model.joblib')


['employee_satisfaction_model.joblib']

In [15]:
model = load('employee_satisfaction_model.joblib')

# Example data for a new employee
new_employee = {
    'last_evaluation': 0.76,
    'number_project': 3,
    'average_montly_hours': 200,
    'time_spend_company': 3,
    'Work_accident': 0,
    'left': 0,
    'promotion_last_5years': 0,
    'Department': 'sales',
    'salary': 'low'
}

new_employee_df = pd.DataFrame([new_employee])

predicted_satisfaction = model.predict(new_employee_df)
print('Predicted Satisfaction Level:', predicted_satisfaction[0])


Predicted Satisfaction Level: 0.6962575783516813
